# Regression : KNN

## 00 환경준비

### 01.Import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

### 02.Data Loading

In [2]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/boston.csv'
data = pd.read_csv(path)
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


|	변수	|	설명	|
|	----	|	----	|
|	medv	|	타운별 집값(중위수)	|
|	crim	|	범죄율	|
|	zn	|	25,000 평방피트를 초과 거주지역 비율	|
|	indus	|	비소매상업지역 면적 비율	|
|	chas	|	찰스강변 위치(범주 : 강변1, 아니면 0)	|
|	nox	|	일산화질소 농도	|
|	rm	|	주택당 방 수	|
|	age	|	1940년 이전에 건축된 주택의 비율	|
|	dis	|	직업센터의 거리	|
|	rad	|	방사형 고속도로까지의 거리	|
|	tax	|	재산세율	|
|	ptratio	|	학생/교사 비율	|
|	black	|	인구 중 흑인 비율	|
|	lstat	|	인구 중 하위 계층 비율	|


## 10.데이터 이해

## 20.데이터 준비

### 1) 데이터 준비

In [ ]:
data.head()

In [3]:
target = 'medv'
x = data.drop(target, axis=1)
y = data.loc[:, target]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 2022)

### 2) 변수정리

### 3) NA 조치

### 4) 가변수화

### 5) Scaling
KNN 알고리즘을 적용하기 위해서는 스케일링을 해야 합니다.

In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

* 정규화 : Normalization, Min Max 방식
    * 모든 변수(열)의 값을 0 ~ 1 사이 값으로 변환

In [7]:
# 선언
scaler1 = MinMaxScaler()

# train으로 .fit + .transform(적용)
x_train_s1 = scaler1.fit_transform(x_train)

#val은 적용만 
x_val_s1 = scaler1.transform(x_val)

* 표준화 : Standardization
    * 모든 변수(열)의 값을, 평균 = 0, 표준편차 = 1 을 갖도록 변환.
    * 정규분포의 의미는 아니다!

In [11]:
scaler2 = StandardScaler()
x_train_s2 = scaler2.fit_transform(x_train)
x_val_s2 = scaler2.transform(x_val)

* 스케일링을 수행하게 되면 결과가 넘파이 어레이로 저장되므로, 다루기 쉽게(.describe()를 사용하기 위해) 데이터프레임으로 변환합니다.
    * .describe() : 데이터프레임, 시리즈의 기초 통계량을 보여줌
    * list(데이터프레임) : 데이터 프레임의 칼럼이름을 반환.

In [12]:
x_train_s1 = pd.DataFrame(x_train_s1, columns = list(x))
x_train_s2 = pd.DataFrame(x_train_s2, columns = list(x))

In [13]:
x.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [14]:
x_train_s1.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,0.036781,0.123354,0.376519,0.069307,0.330768,0.507123,0.647488,0.289496,0.349548,0.403338,0.614730,0.898950,0.287034
std,0.098676,0.239799,0.253303,0.254290,0.240408,0.140416,0.299283,0.217180,0.366536,0.314076,0.233117,0.236220,0.189457
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000718,0.000000,0.163856,0.000000,0.122407,0.422768,0.384840,0.108567,0.130435,0.170172,0.457447,0.949178,0.137900
50%,0.002424,0.000000,0.281525,0.000000,0.271784,0.485603,0.727128,0.234524,0.173913,0.270554,0.664894,0.987064,0.247103
75%,0.027960,0.200000,0.646628,0.000000,0.466805,0.569159,0.930053,0.428455,0.304348,0.476099,0.808511,0.998128,0.395350
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
x_train_s2.describe()

## 30.모델링

### 1)import

In [10]:
# 모델링용
from sklearn.neighbors import KNeighborsRegressor    

# 회귀모델 평가용
from sklearn.metrics import * # 모든 함수 다 불러오란 뜻

### 2) 모델선언

In [ ]:
model = KNeighborsRegressor(n_neighbors = 3)

### 3) 모델링(학습)

In [ ]:
model.fit(x_train_s1, y_train)

### 4) 검증 : 예측

In [ ]:
pred = model.predict(x_val_s1)

### 5) 검증 : 평가

In [ ]:
# MSE
mean_squared_error( y_val, pred   )

In [ ]:
# RMSE
mean_squared_error(y_val, pred, squared=False)

In [ ]:
# MAE
mean_absolute_error(y_val, pred )

In [ ]:
# MAPE : 평균 오차율
mean_absolute_percentage_error(y_val, pred )

In [ ]:
# 1 - MAPE : 정확도
1 - mean_absolute_percentage_error(y_val, pred )

## 40.KNN의 Hyper Parameter

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

* n_neighbors : k 의 갯수. k가 달라지면 예측결과도 달라고, 성능도 달라집니다!
* metric : 거리계산 방식.
    * euclidean : 유클리디안 거리 :  sqrt(a^2 + b^2)
    * manhattan : 맨하탄거리 : a + b
    * minkowski : 위 두가지 거리계산법에 대한 일반화 식

### 1) n_neighbors
하이퍼파라미터를 지정하여 모델을 생성하고 검증해 봅시다.

* model1 : n_neighbors = 5

In [15]:
model1 = KNeighborsRegressor(n_neighbors = 5)

In [16]:
model1.fit(x_train_s1, y_train)

KNeighborsRegressor()

In [17]:
pred1 = model1.predict(x_val_s1)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  "X does not have valid feature names, but"


In [18]:
mean_squared_error(y_val, pred1, squared=False)

3.8986282152552474

* model2 : n_neighbors = 10


In [20]:
model2 = KNeighborsRegressor(n_neighbors = 10)

In [21]:
model2.fit(x_train_s1, y_train)

KNeighborsRegressor(n_neighbors=10)

In [22]:
pred2 = model2.predict(x_val_s1)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  "X does not have valid feature names, but"


In [23]:
mean_squared_error(y_val, pred2, squared=False)

4.540606981969027

* model3 : n_neighbors = 15


In [25]:
model3 = KNeighborsRegressor(n_neighbors = 15)

In [26]:
model3.fit(x_train_s1, y_train)

KNeighborsRegressor(n_neighbors=15)

In [27]:
pred3 = model3.predict(x_val_s1)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  "X does not have valid feature names, but"


In [28]:
mean_squared_error(y_val, pred3, squared=False)

4.540119584193786

* model1,2,3의 성능을 비교해 봅시다.

In [29]:
print(mean_squared_error(y_val, pred1, squared=False))
print(mean_squared_error(y_val, pred2, squared=False))
print(mean_squared_error(y_val, pred3, squared=False))

3.8986282152552474
4.540606981969027
4.540119584193786


In [30]:
print(mean_absolute_error(y_val, pred1 ))
print(mean_absolute_error(y_val, pred2 ))
print(mean_absolute_error(y_val, pred3 ))

2.583137254901961
3.0966666666666662
3.160261437908497


In [31]:
print(mean_absolute_percentage_error(y_val, pred1 ))
print(mean_absolute_percentage_error(y_val, pred2 ))
print(mean_absolute_percentage_error(y_val, pred3 ))

0.1374251471529376
0.16286029034881302
0.17207080449458503


* 성능을 가장 높이는 k(n_neighbors) 값을 어떻게 찾을 수 있을까요?